<h2> Neural Network Music Generation

**Abstract**

With this project, we set out to create a neural network which could somehow "listen" to music, learn compositional patterns,<br>
and generate its own unique music after proper training. To accomplish this, we used MIDI data, which is a digital representation<br>
of music containing distinct events that communicate information such as note pitch and velocity. The goal was to use this data<br>
to get a neural network to generalize harmonic relationships between notes (in other words, determine what sounds "good") as well<br>
as proper musical timing. Over the course of our work, we created two types of networks: a Recurrent Neural Network and a <br>
General Adversarial Network, in order to see how the two options compare in music generation. Future work on this may include <br>
adding in some method of establishing the basic rules of music theory for the network to abide by.

<h4> Recurrent Neural Networks vs. General Adversarial Networks

Below we present two deep learning methods for generating music: *Recurrent Neural Networks* (RNN) and *General Adversarial Networks* (GAN). We will explore the basic concepts of RNN and GNN to generate music. <br> <br>
But before we begin, we will import some helpul packages and tools to accomplish these tasks. 

In [ ]:
import os
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
from typing import Dict, List, Optional, Sequence, Tuple

#MIIDI file processing tools
import pretty_midi
import seaborn as sns
# import fluidsynth

#Neural Network Tools
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from keras.utils.vis_utils import model_to_dot

#Plotting Tools
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from keras.utils.vis_utils import plot_model

<h3> Recurrent Neural Network (RNN)

- explain what the RNN is trying to do (how it works) (Louis)
                - BRIEF intro to RNN structure
                        - example midi file played
                        - explain wHAT we are trying to do (i.e the losses we are trying to minimize
                                                                                        (pitch/step/duration)
                - load up the trained model
                - show the structure of the trained model
                - generate a new song with the model

<h3> General Adversarial Neural Network (GAN)

To create a General Adversarial Neural Network, we will first create 2 different networks and combine them into one. We will contruct: a **generator** and a **discriminator**. <br><br>
The *generator* is designed to take random noise (an array of random values from -1 to 1) and try to generate an appropriate image. The *discriminator* is trained on both the real and fake (generated) images and trys to label the images as fake (generated images) or real (actual images). <br><br> When we combine the models, the networks will be forced to trade-off loss on one another. So, when the discriminator becomes good at determining between real and fake images, the generator is forced to make more real-looking images. When the generator makes better-looking images, the discriminator must learn to make more careful decisions. 

In [ ]:
#tools for NN and image processing
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from keras.utils.vis_utils import model_to_dot
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
from IPython.display import display
from keras.utils.vis_utils import plot_model

But how do we make music by generating images? We will need to do some unique data transformations!

<h5> Midi Images

To be able to generate music with images, we first have to transform our training music (midi files) to images! To do this, we use the helper functions found [here](https://github.com/mathigatti/midi2img).  <br><br>
Using the python script midi2img.py, we converted midi files into a 106 X 106 image, where the **each row represents a note** (from 21 (A0) to 127 (G9)) and **each column represents the timestep of the song** (1/4 beat per pixel). <br><br>
To understand this more clearly, let's load up some example images...

In [ ]:
def grab_image(img_path):
    img = image.load_img(img_path, color_mode="grayscale", target_size=(106,106))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
#read in midi images
exampleImages = np.concatenate([grab_image('./example_midi_images/%s'%(filename)) for filename in os.listdir("./example_midi_images/") if filename.endswith(".png")])

In [ ]:
for i in range(len(exampleImages)):
    plt.imshow(image.array_to_img(exampleImages[i,:,:,:]))
    plt.show()

**To generate new midi images, we created a GAN network consisting of a generator (to create the new midi images) and discriminator (to determine which midi images are real or fake).**

To train the GAN on your own, uncomment the following cells. If you have limited time and want to load a pretrained model, proceed to the cells down below! The following trains on the example images -- if you wish to see better output, add more midi images to the data folder.

In [ ]:
"""
#get data ready to train
x_train = exampleImages
#all real images have a 1 label
y_train = np.ones((exampleImages.shape[0], 1))
img_rows = x_train.shape[1]
img_cols = x_train.shape[2]
channels = x_train.shape[3]
img_shape = (img_rows, img_cols, channels)
num_classes = len(np.unique(y_train))+1
latent_dim = 100
img_shape
"""

In [ ]:
"""
# Generator
noise = keras.layers.Input(shape=(latent_dim,))

label = keras.layers.Input(shape=(1,), dtype='int32')
label_embedding = keras.layers.Embedding(num_classes, latent_dim)(label)
label_embedding = keras.layers.Flatten()(label_embedding)

generator_input = keras.layers.multiply([noise, label_embedding])

generator_hidden = keras.layers.Dense(128 * 52 * 52, # Reshapable
                                      activation='relu',
                                      input_dim=latent_dim)(generator_input)
generator_hidden = keras.layers.Reshape((52, 52, 128))(generator_hidden)
generator_hidden = keras.layers.BatchNormalization(momentum=0.8)(generator_hidden)
generator_hidden = keras.layers.Conv2DTranspose(64,
                                                kernel_size=(4,4),
                                                strides=(2,2),
                                                activation='relu')(generator_hidden)

generator_hidden = keras.layers.BatchNormalization(momentum=0.8)(generator_hidden)
g_image = keras.layers.Conv2DTranspose(channels, 
                                       kernel_size=(7,7),
                                       padding='same',
                                       activation='tanh')(generator_hidden)

generator = keras.Model([noise, label], g_image, name='Generator')
"""

In [ ]:
"""
# Discriminator
d_image = keras.layers.Input(shape=img_shape)

# Hidden layers
discriminator_hidden = keras.layers.Conv2D(16, kernel_size=(3,3), strides=(2,2),padding='same')(d_image)
discriminator_hidden = keras.layers.LeakyReLU(alpha=0.2)(discriminator_hidden)
discriminator_hidden = keras.layers.Dropout(0.25)(discriminator_hidden)
discriminator_hidden = keras.layers.Conv2D(16, kernel_size=(3,3), strides=(2,2), padding='same')(discriminator_hidden)
#discriminator_hidden = keras.layers.ZeroPadding2D(padding=((0,1),(0,1)))(discriminator_hidden)
discriminator_hidden = keras.layers.LeakyReLU(alpha=0.2)(discriminator_hidden)
discriminator_hidden = keras.layers.Dropout(0.25)(discriminator_hidden)
discriminator_hidden = keras.layers.LeakyReLU(alpha=0.2)(discriminator_hidden)
#discriminator_hidden = keras.layers.Dropout(0.25)(discriminator_hidden)
discriminator_hidden = keras.layers.Flatten()(discriminator_hidden)
#discriminator_hidden = keras.layers.BatchNormalization(momentum=0.8)(discriminator_hidden)
discriminator_hidden = keras.layers.Dropout(0.25)(discriminator_hidden)

target_label = keras.layers.Dense(1, activation=keras.activations.sigmoid)(discriminator_hidden)

# Finalize the model
discriminator = keras.Model(d_image, target_label, name="Discriminator")
discriminator.compile(loss=keras.losses.BinaryCrossentropy(),
                      optimizer=keras.optimizers.Adam(0.0002,0.5),  #ADJUST ME!
                      metrics=[keras.metrics.BinaryAccuracy()])
"""

In [ ]:
"""
# Combined model...

combined_model_inputs = [keras.layers.Input(shape=x[1:]) for x in generator.input_shape]
generator_output = generator(combined_model_inputs)
# Turn off learning for discriminator...
discriminator.trainable = False

target_label = discriminator(generator_output) # g_image = generator output layer
# Combined model now takes generator inputs and has discriminator outputs...
combined = keras.Model(combined_model_inputs,target_label)
combined.summary
combined.compile(loss=keras.losses.BinaryCrossentropy(),
                 optimizer=keras.optimizers.Adam(0.0002,0.5))   #ADJUST ME!
"""

In [ ]:
"""
# Training Parameters
batch_size = 4  #ADJUST ME!
half_batch_size = int(batch_size/2)

# Keep track of discriminatory loss[0], disciminator accuracy[1] and generator loss[2]
history = [[],[],[]]
"""

In [ ]:
"""
%%time
# Perform some number of batches...
batches = 20    #ADJUST ME!
for batch in range(batches):
 # Training the DISCRIMINATOR
 # REAL images
 idx = np.random.randint(0, x_train.shape[0], half_batch_size)
 real_images = x_train[idx]
 image_labels = y_train[idx]

 # FAKE images
 noise = np.random.normal(-1, 1, (half_batch_size, 100))
 sampled_labels = np.ones((half_batch_size, 1))
 generated_images = generator.predict([noise, sampled_labels])

 # Assign the fake images to the "fake class"
 fake_labels = np.zeros(half_batch_size).reshape(-1, 1)

 # Train the discriminator - Two groups
 d_loss_real = discriminator.train_on_batch(real_images, image_labels)
 d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
 
 # "overall" loss/accuracy for the discriminator  
 d_loss = np.add(d_loss_real, d_loss_fake) / 2.0

 # Training the generator...
 noise = np.random.normal(-1, 1, (batch_size, 100))

 # calling these real images - try to fake-out the discriminator.
 #Setting the target as if the generated images were real (1)
 sampled_labels = np.ones((batch_size, 1))
 
 # Train the generator
 g_loss = combined.train_on_batch([noise, sampled_labels], sampled_labels)
 
 history[0] += [d_loss[0]]
 history[1] += [d_loss[1]]
 history[2] += [g_loss]
    
 # progress indicator
 print("\r%d [Disc. Loss: %f, Disc. Accuracy: %.2f%%] [Gen. Loss: %f]"%(batch, d_loss[0], 100*d_loss[1], g_loss), end='')
"""

<h5> Training the GAN Network

To train the model, we loaded 563 of the midi images into an array, and we assigned every single image in our training set a "real image" label of 1. <br><br>
We trained the model over **300** batches.<br> <br>
For every batch:
 1. We generated fake images using our generator by giving it random noise (an array of random values from -1 to 1) and the "real" class label of 1.
 
 2. Once our fake midi images were generated, we assigned a "fake" class label of 0 for each image that we generated.
 
 3. We trained the discriminator on the real images with the real labels and the fake images with the fake labels.
 
 4. Finally, we trained the generator with random noise and a target label of 1 (as if the images are real) as input  to fake off the discriminator. 
 

<h5> The Trained GAN

After training, we now have a model that is capable of generating new images from vectors of random noise.
Because training takes several minutes, we will load an already trained model to demonstrate the midi image creation. 

In [ ]:
!wget -O trained_GAN.zip https://www.dropbox.com/s/qr9b3p1xfjkqzlw/trained_GAN.zip?dl=0

In [ ]:
!unzip trained_GAN.zip

In [ ]:
model = keras.models.load_model('./trained_GAN')

Lets take a look at the GAN's architecture!

In [ ]:
#to see individual generator and discriminator layers/components 
#change expand_nested=False to expand_nested=True
keras.utils.plot_model(model, show_shapes=True, expand_nested=False)

<br><br>We just need the generator from the GAN to make some images, so lets load the generator layer into a seperate variable. 

In [ ]:
generator = model.get_layer(name="Generator")

We generate some images using some randome noise and a "real" class label...

In [ ]:
## Testing
r, c = 10, 10
noise = np.random.normal(-1, 1, (r * c, 100))
print(noise.shape)
sampled_labels = np.ones((100, 1))
# Make some fakes!
generated_images = generator.predict([noise, sampled_labels])
# Plot them..

In [ ]:
plt.imshow(image.array_to_img(generated_images[98,:,:,:]))
plt.show()

and save the image...

In [ ]:
imageSave = image.array_to_img(generated_images[98,:,:,:])

In [ ]:
imageSave = imageSave.save('composition.png')

Now that we have generated and saved a midi image, we can use the other helper python script ([img2midi.py](https://github.com/mathigatti/midi2img)) to convert our fresh midi image into a new composition in the midi file format! If you have downloaded the helper functions, you can create this file by running the following command in the terminal: <br>

```python img2midi.py composition.png```